In [1]:
# HuggingFace 다국어 번역 시스템 with Gradio UI
# Google Colab에서 실행 가능한 코드

# ========================================
# 1. 필요한 라이브러리 설치
# ========================================
!pip install -q transformers torch sentencepiece
!pip install -q gradio
!pip install -q langdetect
!pip install -q plotly
!pip install -q pandas


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 12.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:

# ========================================
# 2. 라이브러리 임포트
# ========================================
import gradio as gr
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    MarianMTModel,
    MarianTokenizer,
    MBartForConditionalGeneration,
    MBart50TokenizerFast,
    M2M100ForConditionalGeneration,
    M2M100Tokenizer,
    pipeline
)
from langdetect import detect, detect_langs
import pandas as pd
import plotly.graph_objects as go
import time
from typing import List, Dict, Tuple
import json
import warnings
warnings.filterwarnings('ignore')

# ========================================
# 3. 다국어 번역 시스템 클래스
# ========================================
class MultilingualTranslationSystem:
    """다양한 번역 모델을 지원하는 통합 번역 시스템"""

    def __init__(self):
        self.models = {}
        self.tokenizers = {}
        self.translation_history = []

        # 지원 언어 목록
        self.supported_languages = {
            'ko': '한국어',
            'en': '영어',
            'ja': '일본어',
            'zh': '중국어',
            'es': '스페인어',
            'fr': '프랑스어',
            'de': '독일어',
            'ru': '러시아어',
            'ar': '아랍어',
            'hi': '힌디어',
            'pt': '포르투갈어',
            'it': '이탈리아어',
            'nl': '네덜란드어',
            'tr': '터키어',
            'vi': '베트남어',
            'th': '태국어',
            'id': '인도네시아어',
            'ms': '말레이어',
            'pl': '폴란드어',
            'uk': '우크라이나어'
        }

        # 모델별 지원 언어 쌍
        self.model_capabilities = {
            'mbart-50': list(self.supported_languages.keys()),
            'm2m100': list(self.supported_languages.keys()),
            'opus-mt': ['en', 'ko', 'ja', 'zh', 'es', 'fr', 'de', 'ru']
        }

        print(" 다국어 번역 시스템 초기화 중...")
        self.load_models()
        print(" 시스템 준비 완료!")

    def load_models(self):
        """번역 모델 로드"""
        try:
            # mBART-50 모델 (50개 언어 지원)
            print(" mBART-50 모델 로드 중...")
            self.models['mbart-50'] = MBartForConditionalGeneration.from_pretrained(
                "facebook/mbart-large-50-many-to-many-mmt"
            )
            self.tokenizers['mbart-50'] = MBart50TokenizerFast.from_pretrained(
                "facebook/mbart-large-50-many-to-many-mmt"
            )

            # M2M100 모델 (100개 언어 지원)
            print(" M2M100 모델 로드 중...")
            self.models['m2m100'] = M2M100ForConditionalGeneration.from_pretrained(
                "facebook/m2m100_418M"
            )
            self.tokenizers['m2m100'] = M2M100Tokenizer.from_pretrained(
                "facebook/m2m100_418M"
            )

        except Exception as e:
            print(f" 모델 로드 오류: {e}")

    def get_language_code(self, lang: str, model_type: str) -> str:
        """모델별 언어 코드 변환"""
        # mBART-50 언어 코드
        mbart_codes = {
            'ko': 'ko_KR', 'en': 'en_XX', 'ja': 'ja_XX', 'zh': 'zh_CN',
            'es': 'es_XX', 'fr': 'fr_XX', 'de': 'de_DE', 'ru': 'ru_RU',
            'ar': 'ar_AR', 'hi': 'hi_IN', 'pt': 'pt_XX', 'it': 'it_IT',
            'nl': 'nl_XX', 'tr': 'tr_TR', 'vi': 'vi_VN', 'th': 'th_TH',
            'id': 'id_ID', 'ms': 'ms_MY', 'pl': 'pl_PL', 'uk': 'uk_UA'
        }

        if model_type == 'mbart-50':
            return mbart_codes.get(lang, 'en_XX')
        else:
            return lang

    def translate_with_mbart(self, text: str, src_lang: str, tgt_lang: str) -> str:
        """mBART-50을 사용한 번역"""
        model = self.models['mbart-50']
        tokenizer = self.tokenizers['mbart-50']

        # 언어 코드 변환
        src_code = self.get_language_code(src_lang, 'mbart-50')
        tgt_code = self.get_language_code(tgt_lang, 'mbart-50')

        # 소스 언어 설정
        tokenizer.src_lang = src_code

        # 토큰화
        encoded = tokenizer(text, return_tensors="pt", max_length=1024, truncation=True)

        # 번역 생성
        generated_tokens = model.generate(
            **encoded,
            forced_bos_token_id=tokenizer.lang_code_to_id[tgt_code],
            max_length=1024,
            num_beams=5,
            length_penalty=1.0,
            early_stopping=True
        )

        # 디코딩
        translation = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
        return translation

    def translate_with_m2m100(self, text: str, src_lang: str, tgt_lang: str) -> str:
        """M2M100을 사용한 번역"""
        model = self.models['m2m100']
        tokenizer = self.tokenizers['m2m100']

        # 소스 언어 설정
        tokenizer.src_lang = src_lang

        # 토큰화
        encoded = tokenizer(text, return_tensors="pt", max_length=1024, truncation=True)

        # 타겟 언어 ID 가져오기
        forced_bos_token_id = tokenizer.get_lang_id(tgt_lang)

        # 번역 생성
        generated_tokens = model.generate(
            **encoded,
            forced_bos_token_id=forced_bos_token_id,
            max_length=1024,
            num_beams=5,
            length_penalty=1.0,
            early_stopping=True
        )

        # 디코딩
        translation = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
        return translation

    def translate(self, text: str, src_lang: str, tgt_lang: str, model_type: str = 'auto') -> Dict:
        """통합 번역 함수"""
        start_time = time.time()

        try:
            # 모델 자동 선택
            if model_type == 'auto':
                if 'mbart-50' in self.models:
                    model_type = 'mbart-50'
                elif 'm2m100' in self.models:
                    model_type = 'm2m100'
                else:
                    return {'error': '사용 가능한 모델이 없습니다.'}

            # 번역 수행
            if model_type == 'mbart-50' and 'mbart-50' in self.models:
                translation = self.translate_with_mbart(text, src_lang, tgt_lang)
            elif model_type == 'm2m100' and 'm2m100' in self.models:
                translation = self.translate_with_m2m100(text, src_lang, tgt_lang)
            else:
                return {'error': f'{model_type} 모델을 사용할 수 없습니다.'}

            # 번역 시간 계산
            translation_time = time.time() - start_time

            # 결과 저장
            result = {
                'source_text': text,
                'translated_text': translation,
                'source_lang': src_lang,
                'target_lang': tgt_lang,
                'model_used': model_type,
                'translation_time': round(translation_time, 2),
                'char_count': len(text),
                'word_count': len(text.split())
            }

            # 히스토리에 추가
            self.translation_history.append(result)

            return result

        except Exception as e:
            return {'error': f'번역 오류: {str(e)}'}

    def detect_language(self, text: str) -> Tuple[str, float]:
        """언어 자동 감지"""
        try:
            detections = detect_langs(text)
            if detections:
                lang = detections[0]
                return lang.lang, lang.prob
        except:
            pass
        return 'unknown', 0.0

    def batch_translate(self, texts: List[str], src_lang: str, tgt_lang: str, model_type: str = 'auto') -> List[Dict]:
        """배치 번역"""
        results = []
        for text in texts:
            result = self.translate(text, src_lang, tgt_lang, model_type)
            results.append(result)
        return results

# ========================================
# 4. Gradio UI 구성
# ========================================
class TranslationUI:
    """Gradio 기반 번역 UI"""

    def __init__(self):
        self.translator = MultilingualTranslationSystem()
        self.setup_ui()

    def translate_text(self, text, src_lang, tgt_lang, model_choice, auto_detect):
        """번역 실행 함수"""
        if not text:
            return "텍스트를 입력해주세요.", "", ""

        # 언어 자동 감지
        if auto_detect:
            detected_lang, confidence = self.translator.detect_language(text)
            src_lang = detected_lang
            detection_info = f"감지된 언어: {detected_lang} (신뢰도: {confidence:.2%})"
        else:
            detection_info = ""

        # 번역 수행
        result = self.translator.translate(text, src_lang, tgt_lang, model_choice.lower())

        if 'error' in result:
            return result['error'], "", ""

        # 번역 정보 생성
        info = f"""
         번역 정보:
        • 모델: {result['model_used']}
        • 소요 시간: {result['translation_time']}초
        • 문자 수: {result['char_count']}자
        • 단어 수: {result['word_count']}개
        {detection_info}
        """

        return result['translated_text'], info, self.create_history_table()

    def batch_translate_text(self, texts, src_lang, tgt_lang, model_choice):
        """배치 번역 실행"""
        if not texts:
            return "텍스트를 입력해주세요."

        # 줄바꿈으로 텍스트 분리
        text_list = [t.strip() for t in texts.split('\n') if t.strip()]

        if not text_list:
            return "유효한 텍스트가 없습니다."

        # 배치 번역
        results = self.translator.batch_translate(text_list, src_lang, tgt_lang, model_choice.lower())

        # 결과 포맷팅
        output = []
        for i, result in enumerate(results, 1):
            if 'error' in result:
                output.append(f"{i}. 오류: {result['error']}")
            else:
                output.append(f"{i}. {result['translated_text']}")

        return '\n'.join(output)

    def create_history_table(self):
        """번역 히스토리 테이블 생성"""
        if not self.translator.translation_history:
            return "번역 기록이 없습니다."

        df = pd.DataFrame(self.translator.translation_history)

        # 최근 10개만 표시
        df = df.tail(10)

        # 컬럼 선택 및 이름 변경
        display_df = df[['source_text', 'translated_text', 'source_lang', 'target_lang', 'translation_time']]
        display_df.columns = ['원문', '번역문', '원문 언어', '번역 언어', '소요 시간(초)']

        # 긴 텍스트 축약
        display_df['원문'] = display_df['원문'].apply(lambda x: x[:30] + '...' if len(x) > 30 else x)
        display_df['번역문'] = display_df['번역문'].apply(lambda x: x[:30] + '...' if len(x) > 30 else x)

        return display_df

    def create_statistics_plot(self):
        """번역 통계 그래프 생성"""
        if not self.translator.translation_history:
            return None

        df = pd.DataFrame(self.translator.translation_history)

        # 언어별 번역 횟수
        lang_counts = df['target_lang'].value_counts()

        fig = go.Figure(data=[
            go.Bar(x=lang_counts.index, y=lang_counts.values)
        ])

        fig.update_layout(
            title="언어별 번역 횟수",
            xaxis_title="언어",
            yaxis_title="횟수",
            height=400
        )

        return fig

    def setup_ui(self):
        """Gradio UI 설정"""

        # 언어 목록
        languages = list(self.translator.supported_languages.keys())
        language_names = [f"{code} ({name})" for code, name in self.translator.supported_languages.items()]

        with gr.Blocks(title=" 다국어 번역 시스템", theme=gr.themes.Soft()) as self.interface:

            # 헤더
            gr.Markdown("""
            #  HuggingFace 다국어 번역 시스템

            ### 지원 기능:
            - 20개 언어 간 자유로운 번역
            - 다양한 최신 번역 모델 (mBART-50, M2M100)
            - 언어 자동 감지
            - 배치 번역
            - 번역 히스토리 및 통계
            """)

            with gr.Tabs():
                # 탭 1: 단일 번역
                with gr.TabItem(" 단일 번역"):
                    with gr.Row():
                        with gr.Column():
                            input_text = gr.Textbox(
                                label="번역할 텍스트",
                                placeholder="번역하고 싶은 텍스트를 입력하세요...",
                                lines=5
                            )

                            with gr.Row():
                                src_lang = gr.Dropdown(
                                    choices=languages,
                                    value="ko",
                                    label="원문 언어",
                                    interactive=True
                                )
                                tgt_lang = gr.Dropdown(
                                    choices=languages,
                                    value="en",
                                    label="번역 언어",
                                    interactive=True
                                )

                            model_choice = gr.Radio(
                                choices=["Auto", "mBART-50", "M2M100"],
                                value="Auto",
                                label="번역 모델 선택"
                            )

                            auto_detect = gr.Checkbox(
                                label="원문 언어 자동 감지",
                                value=False
                            )

                            translate_btn = gr.Button(" 번역하기", variant="primary")

                        with gr.Column():
                            output_text = gr.Textbox(
                                label="번역 결과",
                                lines=5,
                                interactive=False
                            )

                            translation_info = gr.Textbox(
                                label="번역 정보",
                                lines=6,
                                interactive=False
                            )

                    # 예제 입력
                    gr.Examples(
                        examples=[
                            ["안녕하세요. 오늘 날씨가 정말 좋네요.", "ko", "en"],
                            ["Hello, how are you today?", "en", "ko"],
                            ["人工知能は私たちの生活を大きく変えています。", "ja", "ko"],
                            ["La inteligencia artificial está revolucionando el mundo.", "es", "ko"],
                            ["Bonjour, comment allez-vous?", "fr", "ko"]
                        ],
                        inputs=[input_text, src_lang, tgt_lang],
                        label="예제 문장"
                    )

                    # 번역 히스토리
                    history_table = gr.Dataframe(
                        label=" 최근 번역 기록",
                        interactive=False
                    )

                    # 이벤트 연결
                    translate_btn.click(
                        fn=self.translate_text,
                        inputs=[input_text, src_lang, tgt_lang, model_choice, auto_detect],
                        outputs=[output_text, translation_info, history_table]
                    )

                # 탭 2: 배치 번역
                with gr.TabItem(" 배치 번역"):
                    with gr.Row():
                        with gr.Column():
                            batch_input = gr.Textbox(
                                label="번역할 텍스트들 (한 줄에 하나씩)",
                                placeholder="첫 번째 문장\n두 번째 문장\n세 번째 문장...",
                                lines=10
                            )

                            with gr.Row():
                                batch_src_lang = gr.Dropdown(
                                    choices=languages,
                                    value="ko",
                                    label="원문 언어"
                                )
                                batch_tgt_lang = gr.Dropdown(
                                    choices=languages,
                                    value="en",
                                    label="번역 언어"
                                )

                            batch_model = gr.Radio(
                                choices=["Auto", "mBART-50", "M2M100"],
                                value="Auto",
                                label="번역 모델"
                            )

                            batch_translate_btn = gr.Button(" 배치 번역 실행", variant="primary")

                        with gr.Column():
                            batch_output = gr.Textbox(
                                label="배치 번역 결과",
                                lines=10,
                                interactive=False
                            )

                    # 배치 예제
                    gr.Examples(
                        examples=[
                            ["안녕하세요\n감사합니다\n좋은 하루 되세요", "ko", "en"],
                            ["Good morning\nThank you\nHave a nice day", "en", "ja"],
                        ],
                        inputs=[batch_input, batch_src_lang, batch_tgt_lang],
                        label="배치 번역 예제"
                    )

                    batch_translate_btn.click(
                        fn=self.batch_translate_text,
                        inputs=[batch_input, batch_src_lang, batch_tgt_lang, batch_model],
                        outputs=batch_output
                    )

                # 탭 3: 언어 감지
                with gr.TabItem(" 언어 감지"):
                    detect_input = gr.Textbox(
                        label="언어를 감지할 텍스트",
                        placeholder="아무 언어로 텍스트를 입력하세요...",
                        lines=5
                    )

                    detect_btn = gr.Button(" 언어 감지", variant="primary")

                    detect_output = gr.Textbox(
                        label="감지 결과",
                        interactive=False
                    )

                    def detect_lang_ui(text):
                        if not text:
                            return "텍스트를 입력해주세요."

                        lang, confidence = self.translator.detect_language(text)

                        if lang in self.translator.supported_languages:
                            lang_name = self.translator.supported_languages[lang]
                            return f"감지된 언어: {lang_name} ({lang})\n신뢰도: {confidence:.2%}"
                        else:
                            return f"감지된 언어: {lang}\n신뢰도: {confidence:.2%}"

                    detect_btn.click(
                        fn=detect_lang_ui,
                        inputs=detect_input,
                        outputs=detect_output
                    )

                    # 언어 감지 예제
                    gr.Examples(
                        examples=[
                            "これは日本語のテキストです。",
                            "Ceci est un texte en français.",
                            "Este es un texto en español.",
                            "Это текст на русском языке.",
                            "هذا نص باللغة العربية",
                            "这是中文文本。"
                        ],
                        inputs=detect_input,
                        label="언어 감지 예제"
                    )

                # 탭 4: 다중 언어 비교
                with gr.TabItem(" 다중 언어 비교"):
                    compare_input = gr.Textbox(
                        label="번역할 텍스트",
                        placeholder="여러 언어로 번역할 텍스트를 입력하세요...",
                        lines=3
                    )

                    compare_src_lang = gr.Dropdown(
                        choices=languages,
                        value="ko",
                        label="원문 언어"
                    )

                    compare_languages = gr.CheckboxGroup(
                        choices=languages,
                        value=["en", "ja", "zh"],
                        label="번역할 언어들 선택"
                    )

                    compare_btn = gr.Button(" 다중 번역 실행", variant="primary")

                    compare_output = gr.Textbox(
                        label="다중 언어 번역 결과",
                        lines=10,
                        interactive=False
                    )

                    def multi_translate(text, src_lang, target_langs):
                        if not text or not target_langs:
                            return "텍스트와 대상 언어를 선택해주세요."

                        results = []
                        results.append(f" 원문 ({src_lang}):\n{text}\n")
                        results.append("=" * 50)

                        for tgt_lang in target_langs:
                            if tgt_lang != src_lang:
                                result = self.translator.translate(text, src_lang, tgt_lang, 'auto')
                                if 'error' not in result:
                                    lang_name = self.translator.supported_languages[tgt_lang]
                                    results.append(f"\n {lang_name} ({tgt_lang}):")
                                    results.append(result['translated_text'])

                        return '\n'.join(results)

                    compare_btn.click(
                        fn=multi_translate,
                        inputs=[compare_input, compare_src_lang, compare_languages],
                        outputs=compare_output
                    )

                # 탭 5: 번역 통계
                with gr.TabItem(" 통계"):
                    stats_btn = gr.Button(" 통계 업데이트", variant="primary")

                    stats_plot = gr.Plot(label="번역 통계")

                    stats_info = gr.Textbox(
                        label="세션 통계",
                        lines=5,
                        interactive=False
                    )

                    def update_stats():
                        if not self.translator.translation_history:
                            return None, "번역 기록이 없습니다."

                        # 통계 계산
                        total_translations = len(self.translator.translation_history)
                        total_chars = sum(h['char_count'] for h in self.translator.translation_history)
                        avg_time = sum(h['translation_time'] for h in self.translator.translation_history) / total_translations

                        models_used = {}
                        for h in self.translator.translation_history:
                            model = h['model_used']
                            models_used[model] = models_used.get(model, 0) + 1

                        info = f"""
                         세션 통계:
                        • 총 번역 수: {total_translations}회
                        • 총 처리 문자: {total_chars:,}자
                        • 평균 번역 시간: {avg_time:.2f}초
                        • 사용된 모델: {', '.join(f"{k}({v}회)" for k, v in models_used.items())}
                        """

                        return self.create_statistics_plot(), info

                    stats_btn.click(
                        fn=update_stats,
                        outputs=[stats_plot, stats_info]
                    )

            # 푸터
            gr.Markdown("""
            ---
            ### 💡 사용 팁:
            - **Auto 모델**: 자동으로 최적의 모델을 선택합니다
            - **mBART-50**: Facebook의 다국어 번역 모델 (50개 언어)
            - **M2M100**: 100개 언어를 지원하는 Many-to-Many 번역 모델
            - **언어 자동 감지**: 원문 언어를 모를 때 사용하세요

            ### 📌 주의사항:
            - 첫 번역 시 모델 로딩으로 시간이 걸릴 수 있습니다
            - 긴 텍스트는 자동으로 분할 처리됩니다
            - 번역 품질은 언어 쌍과 텍스트 도메인에 따라 다를 수 있습니다
            """)

    def launch(self):
        """Gradio 앱 실행"""
        return self.interface

# ========================================
# 5. 메인 실행
# ========================================
if __name__ == "__main__":


    # UI 생성 및 실행
    ui = TranslationUI()
    app = ui.launch()

    # Gradio 앱 실행
    app.launch(share=True, debug=False)




    ╔════════════════════════════════════════════════════════╗
    ║          🌐 다국어 번역 시스템 with Gradio              ║
    ║                                                        ║
    ║  지원 언어: 한국어, 영어, 일본어, 중국어 등 20개 언어    ║
    ║  지원 모델: mBART-50, M2M100                           ║
    ╚════════════════════════════════════════════════════════╝
    
🚀 다국어 번역 시스템 초기화 중...
📥 mBART-50 모델 로드 중...


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/529 [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

✅ 시스템 준비 완료!
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9a59f7fc85dfb7f336.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)



✅ Gradio 앱이 실행되었습니다!
🌐 브라우저에서 표시된 URL로 접속하세요.
💡 share=True로 설정되어 외부에서도 접속 가능합니다.
